In [1]:
##Import some modules used for this notebook.
import os, sys, shutil
import numpy as np
import itertools as it
import copy as cp
import pickle

In [15]:
##Print a dir_list which is a list of ectracted parameters.
##The parameters are made by ./parameter/MakePickleOfParam.ipynb.
dir_list = np.loadtxt("param_list.txt", dtype = "str")
for i in range(len(dir_list)):
    print("{}. {}".format(i, dir_list[i]))

##Input index-number of param_list.txt in dir_num to select it.
##Load the parameters of host-halo and sub-halo.
dir_num = int(input("\nSelect index>> "))
with open("parameters/" + dir_list[dir_num], mode = "rb") as f:
    param = pickle.load(f)
with open("parameters/host_" + dir_list[dir_num], mode = "rb") as f:
    host_param = pickle.load(f)

##The list of used parameters and host-halo.
param_list = list(param.keys())
mainbranch_list = list(param[param_list[0]].keys())
print(param_list)
print(mainbranch_list)

0. param_1e+6_1e+18_ScaleFactor_pid_Mvir_Rvir_x_vx_MW039_MW038.pickle
1. param_1e+7_1e+18_ScaleFactor_pid_Mvir_Rvir_x_vx_MW039_MW038.pickle
2. param_1e+7_1e+18_ScaleFactor_ID_pid_Mvir_Rvir_x_vx_MW039_MW038.pickle
Select index>> 2
['ScaleFactor', 'ID', 'pid', 'Mvir', 'Rvir', 'x', 'vx']
['mainbranch_MW039.csv', 'mainbranch_MW038.csv']


In [23]:
##Get zacc of each sub-halo as acc_sf(Data type is dictionary).
##The zacc is a time when ID of host-halo and pid of sub-halo match.
acc_sf = {}
for m_key in mainbranch_list:
    acc_sf[m_key] = []
    for idx, parameter in enumerate(param["pid"][m_key]):
        for i in range(len(parameter)):
            host_i = len(host_param["ID"][m_key]) - len(parameter) + i
            if host_param["ID"][m_key][host_i] == parameter[i]:
                acc_sf[m_key].append(i)
                break
            acc_sf[m_key].append(-1)

In [24]:
##Describe variables of learning-parameter of a DeepLearningModel in LearnParam.
class LearnParam:
    def __init__(self):
        self.eps = 1.0  ##Prevent devision by zero like 1 / (0 + eps).
        self.bias = True ##If This bias is True, add normalized dataset to bias.
        self.hidden = [[100]*5]  ##Defines number of hidden layers and neurons in each layers.
        self.batch_denominator = [200]  ##Defines a size of mini-batch as size(train-dataset) /  batch_denominator.
        self.learn_rate = ["1e-3"]  ##Learning rate of Back propagation.
        self.optimizer = ["Adam"]  ##Optimizer of the DeepLearningModel.
        self.batch_normalization = [True]  ##Whether to use BatchNormalization or not.
        self.loss_func = ["MSE_RE"]  ##Loss function of a DeepLearning model.
        self.activation_func = ["relu"]  ##Activation function of the hidden layers.
        self.weight_init = ["he"]  ##Condition for initializing weight of the hidden layers.
        self.epoch = 200  ##Training epoch.
        self.input_size = 2  ##Size of input-dataset(axis == 1)
        self.output_size = 7  ##Size of output-dataset(axis == 1)
        self.param_kind = "x"  ##Learning parameter.
        self.train_ratio = 0.7  ##Percentage of training-dataset to all-dataset.
        self.normalize_format = "None"  ##Selects a format of dataset normalization in None, Normalization, Standardization.
        self.extract_dataset = "All"  ##Selects a condition of dataset.
        ##All is overall, Before/After_acc is in part of before/after accretion, All(_not)_acc is overall of (not)accreted sub-halo.
        self.learn_num = 1  ##Distinguishes the directories where training results are saved.
        self.save_fig_type = ".png"  ##Format of the saved figures.

In [27]:
##Extract the data to be used as dataset(data type is double-dictionary).
##Data structure of the dataset is that axis1 is host-halo(ex:MW039, MW038), axis2 is sub-halo.
def make_dataset(m_list, p_kind, h_param, param, ext_data, acc_sf):
    dataset = {}
    for m_key in m_list:
        dataset[m_key] = []
        for idx, parameter in enumerate(param[p_kind][m_key]):
            ##If the data used is after accretion(ext_data == After_Acc), extract after accretion point.
            ##The accretion point is taken from the acc_sf as start_i.
            start_i = 0
            if ext_data in ["After_acc", "All_acc"]:
                if acc_sf[m_key][idx] == -1:
                    ##The acc_sf == -1 is not accretion so skip this halo.
                    continue
                elif ext_data == "After_acc":
                    start_i = acc_sf[m_key][idx]
            if p_kind in ["x", "y", "z"]:
                ##If p_kind is coordinate, use relative value of sub-halo to host-halo.
                data = np.array(parameter[start_i:])
                host_i = data.size
                data -= np.array(h_param[p_key][m_key][-host_i:])
                dataset[m_key].append(data)
            else:
                data = parameter[start_i:]
    return dataset

In [ ]:
class Normalization:
    def __init__(self, m_list, norm_format, eps):
        self.m_list = m_list
        self.norm_format = norm_format
        self.eps = eps

In [ ]:
LP = LearnParam()
dataset = make_dataset(mainbranch_list, LP.param_kind, host_param, param, LP.extract_dataset, acc_sf)